# 文档解析开发指南

阶跃星辰文档解析接口提供强大的文档解析能力，支持 PDF、Word 等多种格式，帮助开发者快速解析文档内容，并将其作为输入用于生成和推理。该接口提供从文档内容提取到生成应用的一站式解决方案，让基于文档内容的应用开发更加高效便捷。

## 效果

- **上传文件**：提供文档上传接口，支持多种文档格式，简化了基于文档的内容的输入。（注意：目前仅支持纯文本内容，图片或扫描形式的文本内容暂不支持）
- **查询文件状态**：文档状态查询接口让开发者及时跟踪解析进度，随时了解文档是否成功解析。
- **读取文件内容**：文档解析成功后，内容提取接口可快速获得文档内容，为后续基于文档内容的生成或分析提供数据基础。
- **查询文件清单**：提供文件清单查询接口，帮助开发者获取已上传文档的列表，便于文档管理。
- **删除文件**：支持文件删除功能，让开发者可以在文档处理完成后随时删除文档，方便管理文档存储。




## 开发步骤

![doc-parser-flow](https://platform.stepfun.com/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fdoc-parser-flow.bcecd2cb.jpg&w=3840&q=75)

通过【上传文件】【查询文件状态】【提取文件内容】+文本补全接口 即可获得完整基于文档内容的对话补全能力

### 1. 上传文件

上传文件到阶跃星辰文件服务，由阶跃星辰文件服务解析文件内容，用于后续生成和推理。

- 上传时使用的 purpose 字段的值为 `file-extract`，用于文件解析。

In [1]:
import requests
import json
import os

# 设置API密钥和文件路径
STEP_API_KEY = os.getenv("STEPFUN_API_KEY") 
file_path = './media/08_step-audio2.pdf'  # 替换为您的文件路径

# 设置请求头和URL
headers = {
    'Authorization': f'Bearer {STEP_API_KEY}'
}
url = 'https://api.stepfun.com/v1/files'

# 准备文件和请求参数
files = {
    'file': open(file_path, 'rb')
}
data = {
    'purpose': 'file-extract'  # 指定用途为文件解析
}

# 发送请求
response = requests.post(url, headers=headers, files=files, data=data)

# 打印响应结果
print(json.dumps(response.json(), indent=2))


{
  "id": "file-Lc7s6mAn9U",
  "object": "file",
  "bytes": 872404,
  "created_at": 1761215390,
  "filename": "08_step-audio2.pdf",
  "purpose": "file-extract"
}


返回结果示例:

```json
{
  "id": "file-abc123",
  "object": "file",
  "bytes": 12345,
  "created_at": 1677610602,
  "filename": "example.pdf",
  "purpose": "file-extract",
  "status": "processing"
}
```

### 2. 查询文件状态

在文件上传完成后，需要查询文件解析状态，确认文件解析是否完成，以便后续获取文件内容。

In [2]:
import requests
import json
import time

# 设置API密钥和文件ID
STEP_API_KEY = os.getenv("STEPFUN_API_KEY") 
file_id = 'file-Lc7s6mAn9U'  # 替换为上传文件后获得的文件ID

# 设置请求头和URL
headers = {
    'Authorization': f'Bearer {STEP_API_KEY}'
}
url = f'https://api.stepfun.com/v1/files/{file_id}'

# 轮询检查文件状态，直到处理完成
max_retries = 10
retry_count = 0

while retry_count < max_retries:
    response = requests.get(url, headers=headers)
    result = response.json()
    print(f'检查次数 {retry_count + 1}, 状态: {result.get("status", "unknown")}')
    
    # 如果文件处理完成，跳出循环
    if result.get('status') == 'success':
        print('文件处理完成！')
        print(json.dumps(result, indent=2))
        break
    
    # 如果文件处理失败，跳出循环
    if result.get('status') == 'error':
        print('文件处理失败！')
        print(json.dumps(result, indent=2))
        break
    
    # 等待一段时间后再次检查
    retry_count += 1
    time.sleep(2)  # 等待2秒

if retry_count >= max_retries:
    print('达到最大重试次数，文件可能仍在处理中。')


检查次数 1, 状态: success
文件处理完成！
{
  "id": "file-Lc7s6mAn9U",
  "object": "file",
  "bytes": 872404,
  "created_at": 1761215391,
  "filename": "08_step-audio2.pdf",
  "purpose": "file-extract",
  "status": "success"
}


返回结果示例，当status为success时，表示文件解析处理完成:

```json
{
  "id": "file-abc123",
  "object": "file",
  "bytes": 12345,
  "created_at": 1677610602,
  "filename": "example.pdf",
  "purpose": "file-extract",
  "status": "success"
}
```

### 3. 提取文件内容

当文件完成解析后，就可以使用提取文件内容接口获取文件内容，并在 Chat API 中，将内容作为对话的输入，生成对话补全。

In [ ]:
import requests
import json

# 设置API密钥和文件ID
STEP_API_KEY = os.getenv("STEPFUN_API_KEY") 
file_id = 'file-Lc7s6mAn9U'  # 替换为您的文件ID

# 设置请求头和URL
headers = {
    'Authorization': f'Bearer {STEP_API_KEY}'
}
url = f'https://api.stepfun.com/v1/files/{file_id}/content'

# 发送请求
response = requests.get(url, headers=headers)

print(response.text)



Step-Audio 2 Technical Report
StepFun Audio Team
Abstract
This paper presents Step-Audio 2, an end-to-end multi-modal large language modeldesigned for industry-strength audio understanding and speech conversation. Byintegrating a latent audio encoder and reasoning-centric reinforcement learning(RL), Step-Audio 2 achieves promising performance in automatic speech recognition(ASR) and audio understanding. To facilitate genuine end-to-end speech conversa-tion, Step-Audio 2 incorporates the generation of discrete audio tokens into languagemodeling, significantly enhancing its responsiveness to paralinguistic informationsuch as speaking styles and emotions. To effectively leverage the rich textual andacoustic knowledge in real-world data, Step-Audio 2 integrates retrieval-augmentedgeneration (RAG) and is able to call external tools such as web search to mitigatehallucination and audio search to switch timbres. Trained on millions of hours ofspeech and audio data, Step-Audio 2 delivers intel

### 4. 基于文档推理完整示例

当提取到需要的文档内容后，可以将文档内容作为对话的输入，生成对话补全。

In [9]:
import requests
import json
import time

# 设置API密钥
STEP_API_KEY = os.getenv("STEPFUN_API_KEY") 
file_path = './media/08_step-audio2.pdf'  # 替换为您的文件路径

# 设置请求头
headers = {
    'Authorization': f'Bearer {STEP_API_KEY}',
    'Content-Type': 'application/json'
}

# 步骤1: 上传文件
def upload_file(file_path):
    upload_url = 'https://api.stepfun.com/v1/files'
    files = {
        'file': open(file_path, 'rb')
    }
    data = {
        'purpose': 'file-extract'
    }
    response = requests.post(upload_url, headers={'Authorization': f'Bearer {STEP_API_KEY}'}, files=files, data=data)
    return response.json()

# 步骤2: 检查文件状态
def check_file_status(file_id):
    status_url = f'https://api.stepfun.com/v1/files/{file_id}'
    response = requests.get(status_url, headers=headers)
    return response.json()

# 步骤3: 获取文件内容
def get_file_content(file_id):
    content_url = f'https://api.stepfun.com/v1/files/{file_id}/content'
    response = requests.get(content_url, headers=headers)
    return response.text

# 步骤4: 使用文件内容进行对话补全
def chat_completion(file_content, question):
    chat_url = 'https://api.stepfun.com/v1/chat/completions'
    payload = {
        'model': 'step-2-mini',  # 使用阶跃星辰的模型
        'messages': [
            {
                'role': 'system',
                'content': '你是一个专业的文档分析助手，请基于提供的文档内容回答问题。'
            },
            {
                'role': 'user',
                'content': f'文档内容：{file_content}问题：{question}'
            }
        ],
        'temperature': 0.7
    }
    response = requests.post(chat_url, headers=headers, json=payload)
    return response.json()

# 执行完整流程
try:
    # 1. 上传文件
    print('正在上传文件...')
    upload_result = upload_file(file_path)
    file_id = upload_result.get('id')
    print(f'文件上传成功，ID: {file_id}')
    
    # 2. 检查文件状态
    print('正在检查文件处理状态...')
    max_retries = 10
    retry_count = 0
    
    while retry_count < max_retries:
        status_result = check_file_status(file_id)
        status = status_result.get('status')
        print(f'文件状态: {status}')
        
        if status == 'success':
            print('文件处理完成！')
            break
        elif status == 'error':
            print('文件处理失败！')
            print(json.dumps(status_result, indent=2))
            break
        
        retry_count += 1
        time.sleep(2)  # 等待2秒后再次检查
    
    if status != 'success':
        print('文件未能成功处理，无法继续。')
        exit()
    
    # 3. 获取文件内容
    print('正在获取文件内容...')
    file_content = get_file_content(file_id)
    content_preview = file_content[:200] + '...' if len(file_content) > 200 else file_content
    print(f'文件内容预览: {content_preview}')
    
    # 4. 使用文件内容进行对话补全
    question = '请总结这份文档的主要内容'  # 您可以替换为任何问题
    print(f'提问: {question}')
    print('正在生成回答...')
    chat_result = chat_completion(file_content, question)
    
    # 提取并显示回答
    answer = chat_result['choices'][0]['message']['content']
    # answer = chat_result.get('choices', [{}])[0].get('message', {}).get('content', '无法获取回答')
    print(f'回答:{answer}')
    
except Exception as e:
    print(f'发生错误: {str(e)}')


正在上传文件...
文件上传成功，ID: file-Lc8fPai7zU
正在检查文件处理状态...
文件状态: processed
文件状态: success
文件处理完成！
正在获取文件内容...
文件内容预览: Step-Audio 2 Technical Report
StepFun Audio Team
Abstract
This paper presents Step-Audio 2, an end-to-end multi-modal large language modeldesigned for industry-strength audio understanding and speech ...
提问: 请总结这份文档的主要内容
正在生成回答...
回答:这份文档是关于Step-Audio 2的研究报告，Step-Audio 2是一个专为工业级音频理解和语音对话设计的端到端多模态大型语言模型。文档主要介绍了Step-Audio 2的设计理念、技术实现、训练方法、评估结果等内容。

1. **设计理念**：Step-Audio 2旨在通过整合潜在音频编码器和以推理为中心的强化学习（RL），实现对语音和音频的理解。同时，通过将离散音频标记的生成整合到语言建模中，实现了真正的端到端语音对话，并提高了对非语言信息（如说话风格和情感）的响应能力。

2. **技术实现**：Step-Audio 2由音频编码器、音频适配器、LLM解码器和音频解tokenizer组成。音频编码器预训练在各种语音和音频理解任务上，包括自动语音识别（ASR）、说话人年龄和性别预测、音频事件检测等。音频适配器的作用是将音频编码器的输出下采样到适当的速率，以便与LLM解码器连接。LLM解码器直接将潜在音频特征作为输入，并输出交错的离散文本和音频标记序列。音频解tokenizer的作用是将音频标记转换为波形。

3. **训练方法**：Step-Audio 2采用多阶段训练策略，在6800亿文本标记和800万小时的合成音频数据上进行训练。训练过程包括ASR数据预训练、音频标记整合、主要预训练和微调等阶段。

4. **评估结果**：Step-Audio 2在各种语音和音频任务上取得了最先进的性能，包括ASR、音频理解、语音翻译和一般语音对话。它在多个语言的ASR测试集上取得了最低的字

## 其他操作实例

### 查看文件列表

In [10]:
import requests
import json

# 设置API密钥
STEP_API_KEY = os.getenv("STEPFUN_API_KEY") 

# 设置请求头和URL
headers = {
    'Authorization': f'Bearer {STEP_API_KEY}'
}
url = 'https://api.stepfun.com/v1/files'

# 可选参数
params = {
    'purpose': 'file-extract',  # 按用途筛选
    'limit': 10  # 限制返回数量
}

# 发送请求
response = requests.get(url, headers=headers, params=params)

# 打印响应结果
print(json.dumps(response.json(), indent=2))


{
  "object": "list",
  "data": [
    {
      "id": "file-Lc7s6mAn9U",
      "object": "file",
      "bytes": 872404,
      "created_at": 1761215391,
      "filename": "08_step-audio2.pdf",
      "purpose": "file-extract",
      "status": "success"
    },
    {
      "id": "file-Lc8CQZeqAK",
      "object": "file",
      "bytes": 872404,
      "created_at": 1761215666,
      "filename": "08_step-audio2.pdf",
      "purpose": "file-extract",
      "status": "success"
    },
    {
      "id": "file-Lc8JYiSIkq",
      "object": "file",
      "bytes": 872404,
      "created_at": 1761215762,
      "filename": "08_step-audio2.pdf",
      "purpose": "file-extract",
      "status": "success"
    },
    {
      "id": "file-Lc8MVVqUEq",
      "object": "file",
      "bytes": 872404,
      "created_at": 1761215802,
      "filename": "08_step-audio2.pdf",
      "purpose": "file-extract",
      "status": "success"
    },
    {
      "id": "file-Lc8Oaht2hM",
      "object": "file",
      "bytes": 87

返回结果示例:

```json
{
  "object": "list",
  "data": [
    {
      "id": "file-abc123",
      "object": "file",
      "bytes": 12345,
      "created_at": 1677610602,
      "filename": "example.pdf",
      "purpose": "file-extract",
      "status": "success"
    },
    {
      "id": "file-def456",
      "object": "file",
      "bytes": 67890,
      "created_at": 1677610702,
      "filename": "another.docx",
      "purpose": "file-extract",
      "status": "success"
    }
  ]
}
```

### 删除文件

In [11]:
import requests
import json

# 设置API密钥和文件ID
STEP_API_KEY = os.getenv("STEPFUN_API_KEY") 
file_id = 'file-Lc7s6mAn9U'  # 替换为您要删除的文件ID

# 设置请求头和URL
headers = {
    'Authorization': f'Bearer {STEP_API_KEY}'
}
url = f'https://api.stepfun.com/v1/files/{file_id}'

# 发送删除请求
response = requests.delete(url, headers=headers)

# 打印响应结果
print(json.dumps(response.json(), indent=2))


{
  "id": "file-Lc7s6mAn9U",
  "object": "file",
  "deleted": true
}


返回结果示例:

```json
{
  "id": "file-abc123",
  "object": "file",
  "deleted": true
}
```

### 删除全部文件

In [12]:
import os
import requests
import json

# 从环境变量读密钥
STEPFUN_API_KEY = os.getenv("STEPFUN_API_KEY")

BASE_URL = "https://api.stepfun.com/v1/files"
HEADERS  = {"Authorization": f"Bearer {STEPFUN_API_KEY}"}

def list_files(purpose="file-extract", limit=100):
    """返回文件列表 [{id, filename, purpose, ...}, ...]"""
    params = {"purpose": purpose, "limit": limit}
    resp = requests.get(BASE_URL, headers=HEADERS, params=params)
    resp.raise_for_status()
    files = resp.json().get("data", [])
    print(f"【共发现 {len(files)} 个文件】")
    for f in files:
        print("  -", f["id"], f.get("filename", ""))
    return files

def delete_file(file_id):
    """删单个文件"""
    url = f"{BASE_URL}/{file_id}"
    resp = requests.delete(url, headers=HEADERS)
    if resp.status_code == 200:
        print(f"✅ 已删除 {file_id}")
    else:
        print(f"❌ 删除 {file_id} 失败", resp.status_code, resp.text)

files = list_files(purpose="file-extract", limit=100)
if not files:
    print("没有需要删除的文件")
else:
    print("\n【开始批量删除】")
    for f in files:
        delete_file(f["id"])
    print("【全部删除完成】")

【共发现 5 个文件】
  - file-Lc8CQZeqAK 08_step-audio2.pdf
  - file-Lc8JYiSIkq 08_step-audio2.pdf
  - file-Lc8MVVqUEq 08_step-audio2.pdf
  - file-Lc8Oaht2hM 08_step-audio2.pdf
  - file-Lc8fPai7zU 08_step-audio2.pdf

【开始批量删除】
✅ 已删除 file-Lc8CQZeqAK
✅ 已删除 file-Lc8JYiSIkq
✅ 已删除 file-Lc8MVVqUEq
✅ 已删除 file-Lc8Oaht2hM
✅ 已删除 file-Lc8fPai7zU
【全部删除完成】


## 注意事项

- **文件解析流程**:
  - 对于首次解析的文件，需先查询文件解析状态，确认成功后再获取内容。
  - 超长文本解析时，请先计算内容的 tokens 数量，以避免超出限制导致的请求中断。

- **文件与请求限制**:
  - 单文件大小：最大 64MB。
  - 文件总数限制：最多存储 1000 个文件。
  - 支持的意图：
    - file-extract：用于文档内容解析。
    - retrieval：用于知识库存储。

- **支持的文件类型**:
  - 纯文本文件：.txt, .md
  - PDF 文件：.pdf
  - Word 文档：.doc, .docx
  - Excel 表格：.xls, .xlsx
  - PPT 文件：.ppt, .pptx
  - CSV 文件：.csv
  - HTML/XML 文件：.html, .htm, .xml

- **内容限制**:
  - 仅支持纯文本内容解析，不支持图片或扫描的文本内容。